In [1]:
import category_encoders as ce
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import shuffle
import math
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
X = pd.read_csv('Data/Hospital_Inpatient_Discharges__SPARCS_De-Identified___20153.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = shuffle(X)
X = df[:(int(df.shape[0]/200))]
X.shape

(11733, 37)

In [4]:
X["Total Costs"] = X["Total Costs"].str.replace('$', '')
X["Total Charges"] = X["Total Charges"].str.replace('$', '')
X["Total Costs"] = X["Total Costs"].str.replace(',', '')
X["Total Charges"] = X["Total Charges"].str.replace(',', '')
X["Total Charges"] = X["Total Charges"].astype(float)
X["Total Costs"] = X["Total Costs"].astype(float)
X["Length of Stay"] = X["Length of Stay"].str.replace("+", "")
X["Length of Stay"] = X["Length of Stay"].astype(float)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [5]:
X.drop(['APR Severity of Illness Description'], axis=1, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [6]:
new_read_data = X.select_dtypes(include=[object])
new_read_data.drop(['APR Risk of Mortality'], axis=1, inplace = True)

In [7]:
ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
X_ohe = ohe.fit_transform(new_read_data)
X_ohe.head()

,Health Service Area_Hudson Valley,Health Service Area_Western NY,Health Service Area_Long Island,Health Service Area_New York City,Health Service Area_nan,Health Service Area_Central NY,Health Service Area_Capital/Adiron,Health Service Area_Finger Lakes,Health Service Area_Southern Tier,Hospital County_Orange,...,Payment Typology 3_Medicare,Payment Typology 3_Private Health Insurance,Payment Typology 3_Federal/State/Local/VA,Payment Typology 3_Unknown,"Payment Typology 3_Managed Care, Unspecified",Payment Typology 3_Miscellaneous/Other,Abortion Edit Indicator_N,Abortion Edit Indicator_Y,Emergency Department Indicator_N,Emergency Department Indicator_Y
704705,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
82312,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2083359,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1680565,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2275025,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [8]:
y = X['Length of Stay']
y.fillna(0, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [9]:
X['APR Risk of Mortality'].replace('Minor', 1, inplace=True)
X['APR Risk of Mortality'].replace('Moderate', 2, inplace=True)
X['APR Risk of Mortality'].replace('Major', 3, inplace=True)
X['APR Risk of Mortality'].replace('Extreme', 4, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
X_ohe['APR Risk of Mortality'] = pd.Series(np.array(X['APR Risk of Mortality']), index=X_ohe.index)
X_ohe['Total Costs'] = pd.Series(np.array(X['Total Costs']), index=X_ohe.index)
X_ohe['Total Charges'] = pd.Series(np.array(X['Total Charges']), index=X_ohe.index)
X_ohe['APR Severity of Illness Code'] = pd.Series(np.array(X['APR Severity of Illness Code']), index=X_ohe.index)

In [11]:
X_ohe.fillna(0,inplace=True)

In [12]:
X_ohe = X_ohe.values
y = y.values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size = 0.20)

In [14]:
X_train.shape

(9386, 1184)

In [ ]:
Bestsvclassifier= SVC(kernel='rbf')
Bestsvclassifier.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
print(np.mean(scores))